# Profile Earth Engine computation

The Earth Engine API provides tools for profiling the performance of your computations but they are not always the easiest to use to get the number you are looking for. The `geetools` library supercharge the original profiler to make any computation evaluation the easiest possible.

In [ ]:
import ee, geetools
from geetools.utils import initialize_documentation

initialize_documentation()

[![github](https://img.shields.io/badge/-see%20sources-white?logo=github&labelColor=555)](https://github.com/gee-community/geetools/blob/main/docs/usage/profile.ipynb)
[![colab](https://img.shields.io/badge/-open%20in%20colab-blue?logo=googlecolab&labelColor=555)](https://colab.research.google.com/github/gee-community/geetools/blob/main/docs/usage/profile.ipynb)

## Set up environment

Install all the requireed libs if necessary. and perform the import satements upstream.

In [ ]:
# uncomment if installation of libs is necessary
# !pip install earthengine-api geetools

In [ ]:
import ee
import geetools
import pandas as pd

In [ ]:
# uncomment if authetication to GEE is needed
# ee.Authenticate()
# ee.Intialize(project="<your_project>")

## Example data

The following examples rely on a `ee.FeatureCollection` composed of three ecoregion features that define regions by which to reduce image data. The Image data are PRISM climate normals, where bands describe climate variables per month; e.g., July precipitation or January mean temperature.


In [ ]:
ecoregions = (
    ee.FeatureCollection("projects/google/charts_feature_example")
    .select(["label", "value","warm"])
)

normClim = ee.ImageCollection('OREGONSTATE/PRISM/Norm91m').toBands()

## default profiler 

The default profiler from Earth Engine can be called as a context manager, it will print at the end of the cell the extensive description of your computation. 

In [ ]:
with ee.profilePrinting():
    normClim.geetools.byBands(
        regions = ecoregions,
        reducer = "mean",
        scale = 500,
        regionId = "label",
        bands = [f"{i:02d}_tmean" for i in range(1,13)],
    ).getInfo()

This result is extremely useful but cannot be further explored in the notebook.

## geetools profiler

The `geetools` profiler is a context manager object that fill a dictionary member (`profile`) with the content of the string profile. This dictionary can be transformed into a table easily.

In [ ]:
# example with a simple function
with ee.geetools.Profiler() as p:
    ee.Number(3.14).add(0.00159).getInfo()
p.profile

With a bigger method we can valorized the results as a pandas dataframe and extract key informations.

In [ ]:
with ee.geetools.Profiler() as p:
    normClim.geetools.byBands(
        regions = ecoregions,
        reducer = "mean",
        scale = 500,
        regionId = "label",
        bands = [f"{i:02d}_tmean" for i in range(1,13)],
    ).getInfo()
df = pd.DataFrame(p.profile)
df.head()

In [ ]:
# total EECU cost of the computation
float(df["EECU-s"].sum())